In [1]:
import re
import subprocess
import os
import json


In [2]:
path = "/Users/user/apps-files/github/cyber_completitions/text-files/"
os.chdir(path)


In [3]:
parsed_commands = []

chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
]


In [4]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    _out = subprocess.run(_args, capture_output=True)
    _text = _out.stdout.decode("utf-8") + _out.stderr.decode("utf-8")

    _lines = _text.split("\n")
    return _lines, command


_lines, command = get_cli_output("cyber")
_lines


['Bostrom Bootloader Hub',
 '',
 'Usage:',
 '  cyber [command]',
 '',
 'Available Commands:',
 '  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query re

In [5]:
get_cli_output("cyber query evidence")


(["Error: invalid evidence hash: encoding/hex: invalid byte: U+002D '-'",
  'Usage:',
  '  cyber query evidence [flags]',
  '',
  'Flags:',
  '      --count-total       count total number of records in evidence to query for',
  '      --height int        Use a specific height to query state at (this can error if the node is pruning state)',
  '  -h, --help              help for evidence',
  '      --limit uint        pagination limit of evidence to query for (default 100)',
  '      --node string       <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
  '      --offset uint       pagination offset of evidence to query for',
  '  -o, --output string     Output format (text|json) (default "text")',
  '      --page uint         pagination page of evidence to query for. This sets offset to a multiple of limit (default 1)',
  '      --page-key string   pagination page-key of evidence to query for',
  '      --reverse           results are sorted in

In [6]:
def parse_chapters(_lines: list, command):

    chapters = {}
    chapters["command"] = command
    chapter = "Title"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


chapters = parse_chapters(_lines, command)
chapters


{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Available Commands': ['  add-genesis-account Add a genesis account to genesis.json',
  '  collect-gentxs      Collect genesis txs and output a genesis.json file',
  '  config              Create or query an application CLI configuration file',
  '  debug               Tool for helping with debugging your application',
  '  export              Export state to JSON',
  '  gentx               Generate a genesis tx carrying a self delegation',
  '  help                Help about any command',
  '  init                Initialize private validator, p2p, genesis, and application configuration files',
  "  keys                Manage your application's keys",
  '  migrate             Migrate genesis to a specified target version',
  '  query               Querying subcommands',
  '  rollback            rollback cosmos-sdk and tendermint state by one height',
  '  start               Run the full node',

In [7]:
def parse_avail_commands(chapters):
    _temp_commands = []

    if "Available Commands" in chapters:

        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                _temp_commands.append(_match.group(1))

        chapters["sub_commands"] = _temp_commands
        del chapters["Available Commands"]
    return chapters


chapters = parse_avail_commands(chapters)
chapters


{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Flags': ['  -h, --help                help for cyber',
  '      --home string         directory for config and data (default "/Users/user/.cyber")',
  '      --log_format string   The logging format (json|plain) (default "plain")',
  '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
  '      --trace               print out full stack trace on errors',
  'Use "cyber [command] --help" for more information about a command.'],
 'sub_commands': ['add-genesis-account',
  'collect-gentxs',
  'config',
  'debug',
  'export',
  'gentx',
  'help',
  'init',
  'keys',
  'migrate',
  'query',
  'rollback',
  'start',
  'status',
  'tendermint',
  'testnet',
  'tx',
  'validate-genesis',
  'version']}

In [8]:
def query_and_parse(command):
    _lines, command = get_cli_output(command)
    commands_dict = parse_chapters(_lines, command)
    commands_dict = parse_avail_commands(commands_dict)
    return commands_dict


commands_dict_list = [query_and_parse("cyber")]
commands_dict_list


[{'command': 'cyber',
  'Title': ['Bostrom Bootloader Hub'],
  'Usage': ['  cyber [command]'],
  'Flags': ['  -h, --help                help for cyber',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors',
   'Use "cyber [command] --help" for more information about a command.'],
  'sub_commands': ['add-genesis-account',
   'collect-gentxs',
   'config',
   'debug',
   'export',
   'gentx',
   'help',
   'init',
   'keys',
   'migrate',
   'query',
   'rollback',
   'start',
   'status',
   'tendermint',
   'testnet',
   'tx',
   'validate-genesis',
   'version']}]

In [9]:
def get_new_commands(chapters: dict):
    new_commands = []
    if "sub_commands" in chapters:
        
        for i in chapters["sub_commands"]:
            command_candidate = " ".join((chapters["command"] + " " + i).split())
            if command_candidate not in parsed_commands:
                new_commands.append(command_candidate)
                parsed_commands.append(command_candidate)

    return new_commands



In [10]:
def loop_commands_list_for_new_comands(commands_dict_list):
    new_commands = []
    for c in commands_dict_list:
        t = get_new_commands(c)
        if t != []:
            new_commands.extend(t)

    return new_commands


In [11]:
all_commands_dict_list = current_dict_list = [query_and_parse('cyber')]

while True:
    new_commands = loop_commands_list_for_new_comands(current_dict_list)
    if new_commands == []:
        break
    current_dict_list = [query_and_parse(command) for command in new_commands]
    all_commands_dict_list.extend(current_dict_list)


In [12]:
# for cyber 0.3 there are 284 commands
assert len(all_commands_dict_list) == 284

284

In [ ]:
all_commands_dict_list

In [14]:
def get_cli_output_and_write(command: str):

    _args = command.split()
    _args.append("-h")

    _out = subprocess.run(_args, capture_output=True)
    _text = _out.stdout.decode("utf-8") + _out.stderr.decode("utf-8")

    file_content = f"### {command}\n\n```\n{_text}\n```\n\n"

    with open("all_commands.md", 'a') as file:
        file.write(file_content)

In [15]:
parsed_sorted = sorted(parsed_commands)
parsed_sorted.insert(0, "cyber")
parsed_sorted


['cyber',
 'cyber add-genesis-account',
 'cyber collect-gentxs',
 'cyber config',
 'cyber debug',
 'cyber debug addr',
 'cyber debug pubkey',
 'cyber debug raw-bytes',
 'cyber export',
 'cyber gentx',
 'cyber help',
 'cyber init',
 'cyber keys',
 'cyber keys add',
 'cyber keys delete',
 'cyber keys export',
 'cyber keys import',
 'cyber keys list',
 'cyber keys migrate',
 'cyber keys mnemonic',
 'cyber keys parse',
 'cyber keys show',
 'cyber migrate',
 'cyber query',
 'cyber query account',
 'cyber query auth',
 'cyber query auth account',
 'cyber query auth accounts',
 'cyber query auth params',
 'cyber query authz',
 'cyber query authz grants',
 'cyber query authz grants-by-grantee',
 'cyber query authz grants-by-granter',
 'cyber query bandwidth',
 'cyber query bandwidth load',
 'cyber query bandwidth neuron',
 'cyber query bandwidth params',
 'cyber query bandwidth price',
 'cyber query bandwidth total',
 'cyber query bank',
 'cyber query bank balances',
 'cyber query bank denom-m

In [16]:
for command in parsed_sorted:
    get_cli_output_and_write(command) 


[(['Bostrom Bootloader Hub',
   '',
   'Usage:',
   '  cyber [command]',
   '',
   'Available Commands:',
   '  add-genesis-account Add a genesis account to genesis.json',
   '  collect-gentxs      Collect genesis txs and output a genesis.json file',
   '  config              Create or query an application CLI configuration file',
   '  debug               Tool for helping with debugging your application',
   '  export              Export state to JSON',
   '  gentx               Generate a genesis tx carrying a self delegation',
   '  help                Help about any command',
   '  init                Initialize private validator, p2p, genesis, and application configuration files',
   "  keys                Manage your application's keys",
   '  migrate             Migrate genesis to a specified target version',
   '  query               Querying subcommands',
   '  rollback            rollback cosmos-sdk and tendermint state by one height',
   '  start               Run the full n